In [3]:

sentdf = pd.read_pickle('sentdf.pkl')

sentdf

,product_parent,discipline,review_body,review_date,star_rating,sentiment
0,598678717,psychology,Review based on a cursory examination by Unive...,2005-10-14,4,4
1,697870179,economy,ALthough he is correct that the IMF and World ...,2005-10-14,3,3
2,339619920,politics,While I don't always agree with his TV analysi...,2005-10-14,2,2
3,733918058,economy,Since the 1992 presidential run of H. Ross Per...,2005-10-14,4,4
4,82232603,economy,In terms of programming concepts and OOP desig...,2005-10-14,5,3
...,...,...,...,...,...,...
42466,218513537,politics,This book captures the reality of the American...,1996-05-19,5,5
42467,426467516,psychology,"Based on anecdotal evidence, intuition, tradit...",1996-05-15,5,4
42468,232298383,politics,An excellent monograph by an outstanding scholar,1995-11-19,5,5
42469,323732312,politics,The authors write with clarity in a field whic...,1995-09-14,4,4


In [4]:
topic = sentdf[['review_body', 'sentiment' , 'product_parent', 'discipline']]

# topic = topic.set_index('review_body')

topic

,review_body,sentiment,product_parent,discipline
0,Review based on a cursory examination by Unive...,4,598678717,psychology
1,ALthough he is correct that the IMF and World ...,3,697870179,economy
2,While I don't always agree with his TV analysi...,2,339619920,politics
3,Since the 1992 presidential run of H. Ross Per...,4,733918058,economy
4,In terms of programming concepts and OOP desig...,3,82232603,economy
...,...,...,...,...
42466,This book captures the reality of the American...,5,218513537,politics
42467,"Based on anecdotal evidence, intuition, tradit...",4,426467516,psychology
42468,An excellent monograph by an outstanding scholar,5,232298383,politics
42469,The authors write with clarity in a field whic...,4,323732312,politics


In [9]:
import contractions

import re
import string
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup

stop=set(stopwords.words('english'))
snow = nltk.stem.SnowballStemmer('english')


def clean_text(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r"n\'t","not",text)
    text = re.sub(r"\'re","are",text)
    text = re.sub(r"\'s","is",text)
    text = re.sub(r"\'d","would",text)
    text = re.sub(r"\'ll","will",text)    
    text = re.sub(r"\'t","not",sentence)
    text = re.sub(r"\'ve","have",sentence)
    text = re.sub(r"\'m","am",sentence)
    text = remove_stopwords(text)
    text = contractions.fix(text)
    return text

preprocessed_reviews=[]


for sentence in topic['review_body'].values:
    sentence=re.sub(r"http\S+"," ",sentence)
    sentence=BeautifulSoup(sentence,'lxml').get_text()
    cleanr=re.compile('<.*?>')
    sentence=re.sub(cleanr,' ',sentence)
    sentence=clean_text(sentence)
    sentence=re.sub("\S\*\d\S*"," ",sentence)
    sentence=re.sub("[^A-Za-z]+"," ",sentence)
    sentence=re.sub(r'[?|!|\'|"|#]',r' ',sentence)
    sentence=re.sub(r'[.|,|)|(|\|/]',r' ',sentence)
    sentence = sentence.lower()
    preprocessed_reviews.append(sentence.strip())

<>:18: DeprecationWarning: invalid escape sequence \w
<>:41: DeprecationWarning: invalid escape sequence \S
<>:18: DeprecationWarning: invalid escape sequence \w
<>:41: DeprecationWarning: invalid escape sequence \S
C:\Users\ibrah\AppData\Local\Temp/ipykernel_2300/2407856499.py:18: DeprecationWarning: invalid escape sequence \w
  text = re.sub('\w*\d\w*', '', text)
C:\Users\ibrah\AppData\Local\Temp/ipykernel_2300/2407856499.py:41: DeprecationWarning: invalid escape sequence \S
  sentence=re.sub("\S\*\d\S*"," ",sentence)


In [37]:
# topic['review_body'] = preprocessed_reviews
# topic.to_pickle('preprocessed.pkl')

# sentdf = pd.read_pickle('preprocessed.pkl')

topic

,review_body,sentiment,product_parent,discipline
0,Review based on a cursory examination by Unive...,4,598678717,psychology
1,ALthough he is correct that the IMF and World ...,3,697870179,economy
2,While I don't always agree with his TV analysi...,2,339619920,politics
3,Since the 1992 presidential run of H. Ross Per...,4,733918058,economy
4,In terms of programming concepts and OOP desig...,3,82232603,economy
...,...,...,...,...
42466,This book captures the reality of the American...,5,218513537,politics
42467,"Based on anecdotal evidence, intuition, tradit...",4,426467516,psychology
42468,An excellent monograph by an outstanding scholar,5,232298383,politics
42469,The authors write with clarity in a field whic...,4,323732312,politics


In [2]:
# LDA WITH GENSIM

# !pip install gensim
# !pip install pyLDAvis
import pickle
import numpy as np
import pandas as pd
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
!pip install pyLDAvis
import spacy
nlp = spacy.load("en_core_web_sm",disable=['parser', 'ner'])
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
nlp.max_length = 45553783
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 're', 'br'])


nlp = spacy.load("en_core_web_sm",disable=['parser', 'ner'])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ibrah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# Convert to list 
data = topic.review_body.tolist()

In [13]:
# Tokenize words and cleanup the text
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['review', 'based', 'on', 'cursory', 'examination', 'by', 'university', 'of', 'phoenix', 'students', 'in', 'philosophy', 'br', 'we', 'have', 'found', 'the', 'book', 'to', 'be', 'well', 'organized', 'and', 'detailed', 'the', 'structure', 'placed', 'the', 'information', 'in', 'an', 'easy', 'to', 'read', 'and', 'presentable', 'manner', 'br', 'the', 'use', 'of', 'strong', 'and', 'sound', 'proofs', 'by', 'experiment', 'and', 'examples', 'supported', 'the', 'overall', 'logic', 'of', 'reasoning', 'in', 'clear', 'and', 'concise', 'manner', 'br', 'we', 'have', 'found', 'the', 'book', 'to', 'be', 'well', 'thought', 'out', 'and', 'laid', 'out', 'so', 'the', 'reader', 'can', 'learn', 'and', 'follow', 'what', 'is', 'being', 'explained', 'br']]


In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['review', 'based', 'on', 'cursory', 'examination', 'by', 'university', 'of', 'phoenix', 'students', 'in', 'philosophy', 'br', 'we', 'have', 'found', 'the', 'book', 'to', 'be', 'well', 'organized', 'and', 'detailed', 'the', 'structure', 'placed', 'the', 'information', 'in', 'an', 'easy', 'to', 'read', 'and', 'presentable', 'manner', 'br', 'the', 'use', 'of', 'strong', 'and', 'sound', 'proofs', 'by', 'experiment', 'and', 'examples', 'supported', 'the', 'overall', 'logic', 'of', 'reasoning', 'in', 'clear', 'and', 'concise', 'manner', 'br', 'we', 'have', 'found', 'the', 'book', 'to', 'be', 'well', 'thought', 'out', 'and', 'laid', 'out', 'so', 'the', 'reader', 'can', 'learn', 'and', 'follow', 'what', 'is', 'being', 'explained', 'br']


In [15]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [17]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['review', 'base', 'cursory', 'examination', 'university', 'phoenix', 'student', 'philosophy', 'find', 'book', 'well', 'organize', 'detailed', 'structure', 'place', 'information', 'easy', 'read', 'presentable', 'manner', 'use', 'strong', 'sound', 'proof', 'experiment', 'example', 'support', 'overall', 'logic', 'reason', 'clear', 'manner', 'find', 'book', 'well', 'think', 'lay', 'reader', 'learn', 'follow', 'explain']]


In [18]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2)]]


In [19]:
# 20 topics anc coherence score:0.5082478433347919
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [20]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.124*"book" + 0.041*"read" + 0.021*"good" + 0.018*"well" + 0.015*"get" + '
  '0.014*"find" + 0.013*"make" + 0.013*"great" + 0.013*"write" + 0.012*"time"'),
 (1,
  '0.141*"woman" + 0.064*"man" + 0.050*"gender" + 0.028*"male" + 0.025*"sex" + '
  '0.023*"feminist" + 0.018*"female" + 0.017*"family" + 0.015*"music" + '
  '0.015*"gay"'),
 (2,
  '0.067*"model" + 0.024*"theory" + 0.019*"mathematical" + 0.016*"analysis" + '
  '0.016*"econometric" + 0.015*"datum" + 0.013*"mathematic" + 0.012*"math" + '
  '0.012*"derivative" + 0.011*"market"'),
 (3,
  '0.063*"political" + 0.034*"politic" + 0.019*"government" + 0.017*"american" '
  '+ 0.013*"public" + 0.012*"war" + 0.012*"medium" + 0.011*"power" + '
  '0.011*"state" + 0.011*"country"'),
 (4,
  '0.038*"history" + 0.024*"reader" + 0.024*"work" + 0.024*"research" + '
  '0.022*"write" + 0.014*"study" + 0.013*"analysis" + 0.012*"essay" + '
  '0.012*"academic" + 0.012*"interested"'),
 (5,
  '0.064*"law" + 0.049*"child" + 0.031*"school" + 0.026*

In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.845060227205302

Coherence Score:  0.5082478433347919


In [27]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.296449 -0.128464       1        1  24.844741
14    -0.260711 -0.107821       2        1  17.155181
15    -0.262008  0.025332       3        1  15.633258
7     -0.236516 -0.032297       4        1  10.043904
3     -0.112181  0.055902       5        1   6.502289
4     -0.165077 -0.135033       6        1   6.054598
6     -0.095923  0.215969       7        1   4.314747
16    -0.088214  0.267243       8        1   3.404891
12     0.023239 -0.143476       9        1   1.656259
2     -0.024496  0.123380      10        1   1.563770
5      0.076516  0.097526      11        1   1.480436
1      0.106466 -0.103841      12        1   1.460840
18     0.120619 -0.121826      13        1   1.377209
13     0.058763 -0.034671      14        1   1.295366
10     0.100958  0.148423      15        1   1.008884
11     0.201002 -0.066909      16        1   0.729272
8      0.203401 -0.048100      17        1   0.482112
19     0.207557 -0.008958      18        1   0.427654
9      0.203162  0.042582      19        1   0.323503
17     0.239893 -0.044960      20        1   0.241087, topic_info=               Term           Freq          Total Category  logprob  loglift
1              book  108898.000000  108898.000000  Default  30.0000  30.0000
48         economic   13827.000000   13827.000000  Default  29.0000  29.0000
24             read   33787.000000   33787.000000  Default  28.0000  28.0000
65        political   14140.000000   14140.000000  Default  27.0000  27.0000
448            life   10790.000000   10790.000000  Default  26.0000  26.0000
...             ...            ...            ...      ...      ...      ...
5083       printing      46.255428      47.548766  Topic20  -5.1605   6.0002
3931  informational      45.318887      46.612225  Topic20  -5.1810   5.9996
1208          cycle     223.496139     301.041149  Topic20  -3.5853   5.7299
8839     millennium      69.491137      85.563458  Topic20  -4.7535   5.8197
8584         foster      79.192355     211.394935  Topic20  -4.6228   5.0459

[1122 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3904     13  0.996139         _
4014     11  0.996039  abortion
1644      6  0.992347  academia
1166      6  0.993959  academic
1166     15  0.005491  academic
...     ...       ...       ...
1163     11  0.192231     young
1163     12  0.283362     young
2170     11  0.871999     youth
2170     12  0.122974     youth
1785     11  0.991547      zone

[2166 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 15, 16, 8, 4, 5, 7, 17, 13, 3, 6, 2, 19, 14, 11, 12, 9, 20, 10, 18])

In [31]:
# 20 topics anc coherence score:0.5082478433347919
lda_model_2 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=13, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [29]:
# Print the keyword of topics
pprint(lda_model_2.print_topics())
doc_lda = lda_model_2[corpus]

[(0,
  '0.073*"book" + 0.034*"read" + 0.016*"make" + 0.014*"get" + 0.014*"good" + '
  '0.011*"think" + 0.011*"time" + 0.011*"know" + 0.011*"people" + '
  '0.010*"well"'),
 (1,
  '0.037*"woman" + 0.026*"man" + 0.017*"life" + 0.015*"child" + 0.014*"human" '
  '+ 0.013*"gender" + 0.010*"family" + 0.008*"society" + 0.008*"self" + '
  '0.007*"live"'),
 (2,
  '0.036*"economic" + 0.020*"theory" + 0.014*"social" + 0.012*"economy" + '
  '0.008*"development" + 0.008*"new" + 0.008*"model" + 0.008*"system" + '
  '0.007*"analysis" + 0.007*"market"'),
 (3,
  '0.024*"political" + 0.012*"politic" + 0.012*"government" + 0.011*"state" + '
  '0.010*"power" + 0.009*"country" + 0.009*"public" + 0.008*"american" + '
  '0.006*"right" + 0.006*"policy"'),
 (4,
  '0.021*"work" + 0.020*"history" + 0.014*"write" + 0.011*"reader" + '
  '0.009*"story" + 0.008*"culture" + 0.008*"modern" + 0.007*"political" + '
  '0.007*"insight" + 0.007*"interested"'),
 (5,
  '0.014*"argument" + 0.014*"fact" + 0.010*"claim" + 0.009*

In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_2.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda_2 = CoherenceModel(model=lda_model_2, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda_2.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.480508907200834

Coherence Score:  0.455135879094558


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_2, corpus, id2word)
vis